# HDF5 Manager - Do Maintenance on Data Files

In [ ]:
using PyPlot, HDF5, DataFrames, Arrow, Printf, Crayons
PyPlot.plt.style.use("./paper.mplstyle")

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)
println(Crayon(foreground = :blue), "In blue. ", Crayon(bold = true), "Red and bold")

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15 
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

In [ ]:
# final time for mean-field
T_final = 10000.

# number of points to get Lyapunov exponent for
npts = 256

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
PATH = "/home/ubuntu/Archives/"

subdir = "small_gaps"
# subdir = "large_gaps"
# subdir = "extra_data"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)

# folder_name = PATH * @sprintf("data/SK_model/N_%i/", N)
instance_names = filter(!isdir, readdir(folder_name; join=true))
# filter!(x -> !occursin("results", x), instance_names)
# filter!(x -> !occursin("undecided", x), instance_names)
# filter!(x -> !occursin("frustrated", x), instance_names)
# filter!(x -> !occursin("late", x), instance_names)
# filter!(x -> !occursin("main_df", x), instance_names);
filter!(x -> occursin("results", x), instance_names);

In [ ]:
length(instance_names)

In [ ]:
test_seed = parse(Int, match(pattern, instance_names[1])[1])

In [ ]:
h5open(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "r")

In [ ]:
# h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "exact_ARPACK_LM_lowest_eigvecs")

## Auxiliary Plotting

### Check degeneracy at final time

In [ ]:
seeds_to_plot = []
for (k, instance_name) in enumerate(instance_names)
    # println(test_seed, k)
    test_seed = parse(Int, match(pattern, instance_name)[1])
    λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "exact_ARPACK_LM_eigvals")

    final_gap = λ[2, end] .- λ[1, end]
    if final_gap < 0.01
        push!(seeds_to_plot, test_seed)
    end
end

In [ ]:
numplots = 16
length(seeds_to_plot)

In [ ]:
start_idx = 1
figure(figsize=(2.5, 2numplots))
# for (k, instance_name) in enumerate(instance_names[start_idx:start_idx-1+numplots])
for (k, test_seed) in enumerate(seeds_to_plot[start_idx:start_idx-1+numplots])
    # println(test_seed, k)
    λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "exact_ARPACK_LM_eigvals")

    final_gap = λ[2, end] .- λ[1, end]
    if final_gap < 0.01
        ax = subplot(numplots, 1, k)
        ax.plot(range(0, 1, 33), λ[2, :] .- λ[1, :], label=string(test_seed) * ", " * @sprintf("%0.3f", final_gap))
        ax.plot(range(0, 1, 33), λ[3, :] .- λ[1, :])
        xlim(0, 1)
        ylim(0, 0.5)
        legend()
    end
end
tight_layout()

### Plot all gaps

In [ ]:
seeds_to_plot = [2339, 2354, 2357, 2390, 2434, 2446, 2508]

In [ ]:
start_idx = 1
figure(figsize=(2.5, length(seeds_to_plot)))
# for (k, instance_name) in enumerate(instance_names[start_idx:start_idx-1+numplots])
for (k, test_seed) in enumerate(seeds_to_plot)
    # println(test_seed, k)
    λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "exact_ARPACK_LM_eigvals")

    ax = subplot(length(seeds_to_plot), 1, k)
    ax.plot(range(0, 1, 33), λ[2, :] .- λ[1, :], label=string(test_seed))
    ax.plot(range(0, 1, 33), λ[3, :] .- λ[1, :])
    xlim(0, 1)
    ax.set_xticklabels([])
    ylim(0, 0.5)
    legend()
end
tight_layout()

In [ ]:
for (k, instance_name) in enumerate(instance_names)
    test_seed = parse(Int, match(pattern, instance_name)[1])
    print(test_seed, ", ", k, "\t")
    λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, test_seed), "exact_ARPACK_LM_eigvals")
    final_gap = λ[2, end] .- λ[1, end]

    figure(figsize=(3, 2))
    plot(range(0, 1, 33), λ[2, :] .- λ[1, :], label=string(test_seed) * ", " * @sprintf("%0.3f", final_gap))
    xlim(0, 1)
    ylim(0, 2)
    legend()
    # savefig(PATH * @sprintf("plots/SK_model/N_%i/%s/", N, subdir) * @sprintf("random_SK_instance_N_%i_seed_%i.pdf", N, test_seed), dpi=128, bbox_inches="tight")
    close()
end

## Deleting data (careful!)

In [ ]:
for instance_name in instance_names
    # print(instance_name, "\n")    
    f = h5open(instance_name, "r+")
    println(keys(f))
end

In [ ]:
# instances
del_keys = ["spectra_T_final_32768_tau_final_2048"]

for instance_name in instance_names
    print(instance_name[53:end], "\n")    
    f = h5open(instance_name, "r+")
    for del_key in del_keys
        if haskey(f, del_key)
            printstyled(@sprintf("key %s", del_key) * "\n", color=:red)
            # delete_object(f, del_key)
        end
    end
end


## Move data

In [ ]:
T_final = 32768.
τ_final = 2048.0

npts_diag = 16
T_diags = T_final .* range(0.5, 1.0, npts_diag + 1)

for instance_name in instance_names
    # print(instance_name, "\n")    
    f = h5open(instance_name, "r+")
    for k in 1:17
        try
            nus = h5read(instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final))
            spec_sum = h5read(instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final))
            h5write(folder_name * "spectrum_" * instance_name[60:end], @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final), 2pi .* nus)
            h5write(folder_name * "spectrum_" * instance_name[60:end], @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final), spec_sum)        
        catch
            println("No spectrum!")
        end
    end
end

In [ ]:
# mv_list = [k for (k, v) in small_minigaps]
# mv_list = [k for (k, v) in large_minigaps]
mv_list = seeds_to_plot

type_str = "data"
file_suffix = "h5"

for seed in mv_list
    try
        # move to small
        # mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
       
        # move to large
        # mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "large_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
        
        mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), 
           PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "degenerate/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))

        mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("results_random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), 
           PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "degenerate/" * @sprintf("results_random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))           
    catch
        print(seed, ", ")
    end
end

#### Move plots

In [ ]:
# # mv_list = [k for (k, v) in small_minigaps]
# mv_list = [k for (k, v) in large_minigaps]

# type_str = "plots"
# file_suffix = "pdf"

# for seed in mv_list
#     try
#         # mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#         mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "large_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#     catch
#         print(seed, ", ")
#     end
# end